# Field Validation

In [3]:
import pandas as pd #reading dataset
import matplotlib as mpl    #plotting, visualization
import matplotlib.pyplot as plt
import tensorflow as tf #building model
from tensorflow.keras.models import Model
import numpy as np  #playing output with model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import joblib  
import json

import sys
sys.path.append('../../')   # Add parent directory to Python path
from utils.Preprocessing import *
from utils.Segmentation import *


In [ ]:
#load processed data
df = pd.read_csv('../../data/test/Accelerometer_ziyi_with_curb_segmented_abnormal_not_ok.csv')

,curb_scene,start_time,end_time,Acc-Z_1,Acc-Z_2,Acc-Z_3,Acc-Z_4,Acc-Z_5,Acc-Z_6,Acc-Z_7,...,Acc-Z_91,Acc-Z_92,Acc-Z_93,Acc-Z_94,Acc-Z_95,Acc-Z_96,Acc-Z_97,Acc-Z_98,Acc-Z_99,Acc-Z_100
0,0,2025-03-28 16:00:48.289,2025-03-28 16:00:48.780,8.374924,8.274368,7.814682,8.034943,7.891296,8.073242,8.173798,...,8.078033,8.020569,8.011002,8.078033,8.020569,8.020569,7.953537,7.991852,8.087616,8.039734
1,0,2025-03-28 16:00:48.785,2025-03-28 16:00:49.277,8.030151,7.977478,8.020569,8.054092,8.020569,8.020569,7.987061,...,7.441177,7.259216,7.584824,7.536942,7.278366,7.254425,7.503433,7.704544,7.781158,7.742844
2,0,2025-03-28 16:00:49.284,2025-03-28 16:00:49.773,7.517792,7.723694,7.939178,7.718903,7.733276,7.508209,7.498642,...,6.689407,6.689407,6.459549,6.234497,5.750870,5.477936,5.722137,6.766022,7.991852,8.384491
3,0,2025-03-28 16:00:49.778,2025-03-28 16:00:50.270,8.633499,9.428360,10.046066,8.925583,7.010223,7.082046,6.885727,...,6.689407,8.609543,9.916779,8.930374,8.427597,8.173798,8.384491,7.412445,7.776367,8.011002
4,0,2025-03-28 16:00:50.275,2025-03-28 16:00:50.766,10.960663,10.577591,9.801865,7.599197,6.493073,8.561661,8.892059,...,9.284714,9.232040,8.437164,8.576035,9.030930,9.514557,9.265564,9.686935,10.113113,9.428360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,1,2025-03-28 16:22:14.798,2025-03-28 16:22:15.291,16.443375,1.996765,-1.283295,19.369095,27.777527,29.927520,12.052414,...,10.840942,14.858414,10.098740,6.947967,7.034164,11.310211,11.171341,12.832916,13.182480,9.763550
2682,1,2025-03-28 16:22:15.297,2025-03-28 16:22:20.198,12.871231,15.298950,12.167328,6.507446,4.898544,10.793060,14.676453,...,4.027054,16.352402,15.236694,1.853119,-0.871490,9.572021,15.729904,10.572800,0.057465,-0.823608
2683,1,2025-03-28 16:22:20.201,2025-03-28 16:22:20.695,10.534485,14.997284,9.686935,1.532288,4.223373,10.481812,11.621460,...,5.468353,8.595184,13.000519,10.860092,9.801865,4.520248,8.724472,13.666107,11.607086,5.535400
2684,1,2025-03-28 16:22:20.698,2025-03-28 16:22:21.190,0.430954,6.354217,10.826584,11.386826,4.587295,1.154007,9.184158,...,16.448166,22.376205,11.731583,1.762131,5.252884,11.281479,13.757080,8.930374,3.931274,14.647720


In [13]:
df['start_time'] = pd.to_datetime(df['start_time'])
df = df.sort_values(by='start_time').reset_index(drop=True)
df

,curb_scene,start_time,end_time,Acc-Z_1,Acc-Z_2,Acc-Z_3,Acc-Z_4,Acc-Z_5,Acc-Z_6,Acc-Z_7,...,Acc-Z_91,Acc-Z_92,Acc-Z_93,Acc-Z_94,Acc-Z_95,Acc-Z_96,Acc-Z_97,Acc-Z_98,Acc-Z_99,Acc-Z_100
0,0,2025-03-28 16:00:48.289,2025-03-28 16:00:48.780,8.374924,8.274368,7.814682,8.034943,7.891296,8.073242,8.173798,...,8.078033,8.020569,8.011002,8.078033,8.020569,8.020569,7.953537,7.991852,8.087616,8.039734
1,0,2025-03-28 16:00:48.785,2025-03-28 16:00:49.277,8.030151,7.977478,8.020569,8.054092,8.020569,8.020569,7.987061,...,7.441177,7.259216,7.584824,7.536942,7.278366,7.254425,7.503433,7.704544,7.781158,7.742844
2,0,2025-03-28 16:00:49.284,2025-03-28 16:00:49.773,7.517792,7.723694,7.939178,7.718903,7.733276,7.508209,7.498642,...,6.689407,6.689407,6.459549,6.234497,5.750870,5.477936,5.722137,6.766022,7.991852,8.384491
3,0,2025-03-28 16:00:49.778,2025-03-28 16:00:50.270,8.633499,9.428360,10.046066,8.925583,7.010223,7.082046,6.885727,...,6.689407,8.609543,9.916779,8.930374,8.427597,8.173798,8.384491,7.412445,7.776367,8.011002
4,0,2025-03-28 16:00:50.275,2025-03-28 16:00:50.766,10.960663,10.577591,9.801865,7.599197,6.493073,8.561661,8.892059,...,9.284714,9.232040,8.437164,8.576035,9.030930,9.514557,9.265564,9.686935,10.113113,9.428360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2681,0,2025-03-28 16:22:59.826,2025-03-28 16:23:00.318,7.527374,7.082046,7.699753,8.097183,3.246536,10.180145,6.402100,...,8.264786,8.092407,8.743622,8.820236,8.465897,7.723694,7.182602,7.675812,8.078033,7.618347
2682,0,2025-03-28 16:23:00.324,2025-03-28 16:23:00.815,6.919251,7.096420,8.125916,8.643066,8.264786,7.790726,7.967911,...,7.843399,7.881714,7.987061,7.958328,7.948746,7.920013,7.872131,8.015793,8.135498,7.929596
2683,0,2025-03-28 16:23:00.821,2025-03-28 16:23:01.311,7.881714,7.838623,7.996628,8.011002,7.862564,7.833832,7.852982,...,7.833832,7.690170,7.733276,7.819458,7.800308,7.675812,7.551315,7.623138,7.704544,7.704544
2684,0,2025-03-28 16:23:01.319,2025-03-28 16:23:01.811,7.580048,7.465118,7.575256,7.637497,7.685394,7.450760,7.489059,...,7.733276,7.776367,7.642288,7.556107,7.642288,7.704544,7.690170,7.608780,7.575256,7.632721


In [21]:
# Convert time string to datetime.time
target_time = pd.to_datetime('16:02:50.023').time()

# Filter rows based on time
df[df['start_time'].dt.time == target_time]

,curb_scene,start_time,end_time,Acc-Z_1,Acc-Z_2,Acc-Z_3,Acc-Z_4,Acc-Z_5,Acc-Z_6,Acc-Z_7,...,Acc-Z_91,Acc-Z_92,Acc-Z_93,Acc-Z_94,Acc-Z_95,Acc-Z_96,Acc-Z_97,Acc-Z_98,Acc-Z_99,Acc-Z_100
246,1,2025-03-28 16:02:50.023,2025-03-28 16:02:50.515,9.591171,12.239166,9.864105,4.132385,3.586517,13.249512,12.627014,...,14.097061,10.012558,6.28717,1.522705,5.817917,4.865021,2.853882,4.888962,6.971909,11.30542


In [24]:
df.iloc[247]

curb_scene                             1
start_time    2025-03-28 16:02:50.519000
end_time         2025-03-28 16:02:51.012
Acc-Z_1                         7.163452
Acc-Z_2                         6.263229
                         ...            
Acc-Z_96                       -3.002335
Acc-Z_97                        2.791641
Acc-Z_98                       10.663773
Acc-Z_99                       18.531128
Acc-Z_100                       7.556107
Name: 247, Length: 103, dtype: object

In [ ]:
#relabel error low data to 0
df_low = [181, 219, 225, 227, 315, 317, 477, 479, 545, 634, 828, 936, 1017, 1065, 1067, 1068, 
          1070, 1091, 1093, 1139, 1199, 1201, 1247, 1249, 1280, 1282, 1308, 1311, 1436, 1439, 1470, 
          1472, 1524, 1589, 1592, 1635, 1683, 1685, 1687, 1715, 1804, 1807, 1950, 1956, 1958, 1983, 2020, 2023, 2025, 2047, 2049, 2094, 2096, 2097, 2100, 2101, 2104, 2106, 2108, 2110, 2112, 2145, 2150, 2153, 2158, 2160, 2163, 2320, 2321, 2322, 2323, 2324, 2326, 2327, 2329, 2330, 2331, 2332, 2333, 2334, 2335, 2338, 2339, 2340, 2341, 2342, 2344, 2345, 2348, 2352, 2353, 2354, 2356, 2357, 2361, 2362, 2364, 2365, 2366, 2368, 2369, 2370, 2371, 2372, 2373, 2374, 2375, 2376, 2377, 2378, 2379, 2380, 2381, 2382, 2383, 2384, 2385, 2386, 2387, 2388, 2389, 2390, 2391, 2392, 2393, 2394, 2395, 2397, 2398, 2401, 2402, 2403, 2404, 2405, 2406, 2407, 2412, 2413, 2414, 2415, 2417, 2418, 2419, 2420, 2421, 2422, 2423, 2425, 2426, 2428, 2429, 2430, 2431, 2433, 2434, 2437, 2438, 2441, 2442, 2445, 2446, 2447, 2449, 2450, 2451, 2452, 2453, 2454, 2455, 2457, 2458, 2459, 2460, 2461, 2462, 2463, 2464, 2465, 2466, 2468, 2469, 2470, 2472, 2473, 2474, 2477, 2478, 2479, 2481, 2482, 2483, 2485, 2486, 2487, 2489, 2491, 2493, 2494, 2495, 2497, 2498, 2499, 2502, 2503, 2504, 2505, 2506, 2507, 2510, 2511, 2512, 2514, 2515, 2517, 2518, 2519, 2520, 2521, 2522, 2523, 2525, 2526, 2527, 2530, 2531, 2533, 2534, 2535, 2538, 2539, 2542, 2545, 2546, 2547, 2548, 2549, 2550, 2551, 2552, 2553, 2554, 2555, 2556, 2557, 2558, 2559, 2560, 2561, 2562, 2565, 2566, 2567, 2568, 2569, 2570, 2571, 2574, 2575, 2577, 2578, 2579, 2581, 2582, 2583, 2584, 2586, 2587, 2590, 2591, 2592, 2593, 2597, 2598, 2599, 2600, 2605, 2606, 2607, 2610, 2615, 2616, 2617, 2618, 2619, 2620, 2621, 2622, 2623, 2625, 2626, 2627, 2630, 2631, 2632, 2635, 2636, 2637, 2639, 2640, 2641, 2642, 2643, 2644, 2647, 2648, 2649, 2650, 2651, 2652, 2653, 2655, 2656, 2657, 2658, 2659, 2660, 2661, 2662, 2663, 2664, 2665, 2667, 2668, 2669, 2670, 2674, 2675, 2676, 2677, 2683, 2684]

# Update curb_scene values to 1 for these indices
df.loc[df_low, 'curb_scene'] = 0

In [19]:
#save the updated DataFrame to a new CSV file
df.to_csv('../../data/test/Accelerometer_ziyi_with_curb_segmented_updated.csv', index=False)